# The objective of this notebook is to extract the names and postal codes of the neighborhoods of Toronto, which are then used to obtain their corresponding longitude and latitude to be used with Foursquare's API to find the venues in these neighborhoods to be clustered according to the most common ones

## The names and postal codes of neighborhoods of Toronto can be found on this wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#### The first step is to install the necessary packages that will be used to parse the html code of wikipedia's page

In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


#### By importing the BeautifulSoup and requests packages, we can use them to access and parse wikipedia's webpage to extract the necessary information

In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
soup = BeautifulSoup(source, 'lxml')

#### By exploring the webpage, we notice that the table that contains the information of interest actually corresponds to the part of the html code that comes after the header table

#### This section cleans the obtained text to extract the important information into a DataFrame as required

In [7]:
# Rmove newlines and '' elements of the parsed text
data = soup.find('table').text.split('\n')
data = [d for d in data if not d == '']
data[:10]

['Postcode',
 'Borough',
 'Neighbourhood',
 'M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A']

In [8]:
header = data[:3] # Captures the header of the table
data = data[3:]
data[:10]

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A']

In [9]:
header

['Postcode', 'Borough', 'Neighbourhood']

In [10]:
# Creates a list of dictionaries where each dictionary corresponds to a row in the table or in the required DataFrame
data_form = []
for i in range(0,len(data),3):
    # Make sure the not assigned Boroughs are dropped
    if not data[i+1] == 'Not assigned':
        # Make sure that not assigned Neighborhoods take the same values as their corresponding Boroughs
        if data[i+2] == 'Not assigned':
            data[i+2] = data[i+1]
        d = dict(zip(header, data[i:i+3]))
        data_form.append(d)

In [11]:
# Create a DataFrame, where the columns titles match the requirement
import pandas as pd

data_df = pd.DataFrame(data_form)
data_df.rename(columns = {'Neighbourhood':'Neighborhood', 'Postcode':'PostalCode'}, inplace=True)
data_df = data_df[['PostalCode', 'Borough', 'Neighborhood']]
data_df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### An important assumption here is that the postal codes are alphabetically ordered

In [12]:
# Ged rid of repeated postal codes by appending the Neighborhoods as required
repeat_ind = []
for i in range(0,len(data_df)-1):
    if data_df.iloc[i,0] == data_df.iloc[i+1,0]:
        data_df.iloc[i+1, 2] = '{}, {}'.format(data_df.iloc[i+1,2],data_df.iloc[i,2])
        repeat_ind.append(i)
data_df.drop(repeat_ind, inplace=True)

In [13]:
# Fix the numbering of the rows after dropping the repeated postal codes
data_df.reset_index(inplace = True)
data_df.drop(columns='index', inplace=True)
data_df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Display the shape of the DataFrame

In [14]:
data_df.shape

(103, 3)

#### Install and import the geocoder python package in order to obtain thelongitude and latitude of each of the postal codes

In [15]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [16]:
import geocoder # import geocoder

#### Define the function to be used to be persistent in obtaining the longitudes and latidues

In [17]:
def obtain_ll(postal_code):

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return [latitude, longitude]

#### Loop over all postal codes to obtain their corresponding latitudes and longitudes (This approach took so much time, so I will use the excel sheet instead)

In [18]:
#long = []
#lat = []
#for i in range(0,len(data_df)):
#    coor = obtain_ll(data_df.iloc[i,0])
#    lat.append(coor[0])
#    long.append(coor[1])
#    print (coor)

In [19]:
import csv
import pandas as pd

In [20]:
# Convert the csv file into a DataFrame
path = r'C:\Users\amt\Downloads\Geospatial_Coordinates.csv'
pd_ll = pd.read_csv(path) 
pd_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
# Extract the longitudes and latitudes corresponding to each postal code
long = []
lat = []
for i in range(0,len(data_df)):
    for j in range(0,len(pd_ll)):
        if data_df.iloc[i,0] == pd_ll.iloc[j,0]:
            long.append(pd_ll.iloc[i,2])
            lat.append(pd_ll.iloc[i,1])
            continue

In [22]:
# Update the table to include columns for the longitudes and latitudes
data_df['Latitude'] = lat
data_df['Longitude'] = long
data_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,43.773136,-79.239476


In [23]:
data_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
7,M3B,North York,Don Mills North,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


In [24]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [26]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [27]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


In [28]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [29]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data_df['Latitude'], data_df['Longitude'], data_df['Borough'], data_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

####  Let's work with only boroughs that contain the word Toronto. So let's slice the original dataframe

In [30]:
# Find the different names of Boroughs we have in the dataset
Toronto_borough = data_df['Borough'].unique()
Toronto_borough

array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [31]:
# Filter the dataframe for the entries that Borough has Toronto in it
list_of_values = ['Downtown Toronto', 'East Toronto', 'West Toronto','Central Toronto']
Toronto_data = data_df[data_df['Borough'].isin(list_of_values)].reset_index(drop=True)
#Toronto_data = data_df[:]
Toronto_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M4E,East Toronto,The Beaches,43.786947,-79.385975
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
5,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
6,M6G,Downtown Toronto,Christie,43.753259,-79.329656
7,M5H,Downtown Toronto,"Richmond, King, Adelaide",43.737473,-79.464763
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944
9,M5J,Downtown Toronto,"Union Station, Toronto Islands, Harbourfront East",43.695344,-79.318389


In [32]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'],\
                                           Toronto_data['Borough'], Toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [33]:
# Define foursquare creditentials and version
CLIENT_ID = 'ZXCLYSYUWVNJPEC5ITUIVXFZXHADNFX2FYFNHZE2GEQLP3H1' # your Foursquare ID
CLIENT_SECRET = 'CPA5PULYPWSCPIVZPPEXQJ2XKRG2CTGNYVNK3IONY2SKJIXI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZXCLYSYUWVNJPEC5ITUIVXFZXHADNFX2FYFNHZE2GEQLP3H1
CLIENT_SECRET:CPA5PULYPWSCPIVZPPEXQJ2XKRG2CTGNYVNK3IONY2SKJIXI


#### we know that all the information is in the items key. 

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### A function to find top 100 venues within a radius of 500 meters for the neighborhood in Toronto

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            continue
            
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
# Run the above function on each neighborhood and create a new dataframe called Toronto_venues
LIMIT = 100
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, King, Adelaide
Dufferin, Dovercourt Village
Union Station, Toronto Islands, Harbourfront East
Trinity, Little Portugal
Riverdale, The Danforth West
Toronto Dominion Centre, Design Exchange
Parkdale Village, Exhibition Place, Brockton
India Bazaar, The Beaches West
Victoria Hotel, Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill West, Forest Hill North
The Junction South, High Park
North Toronto West
Yorkville, North Midtown, The Annex
Roncesvalles, Parkdale
Davisville
University of Toronto, Harbord
Swansea, Runnymede
Summerhill East, Moore Park
Kensington Market, Grange Park, Chinatown
Summerhill West, South Hill, Rathnelly, Forest Hill SE, Deer Park
South Niagara, Railway Lands, King and Spadina, Harbourfront West, Island airport, Bathurst Quay, CN Tower
Rosedale
Stn A PO Boxes 25 The Esplanade
St. James Town, Cabbagetown
Und

In [37]:
# check the size of the resulting dataframe
print(Toronto_venues.shape)
Toronto_venues.head()

(780, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
4,"Regent Park, Harbourfront",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


In [38]:
# check how many venues were returned for each neighborhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,1,1,1,1,1,1
Business Reply Mail Processing Centre 969 Eastern,3,3,3,3,3,3
Central Bay Street,4,4,4,4,4,4
Christie,2,2,2,2,2,2
Church and Wellesley,7,7,7,7,7,7
Davisville,4,4,4,4,4,4
Davisville North,100,100,100,100,100,100
"Dufferin, Dovercourt Village",4,4,4,4,4,4
"Forest Hill West, Forest Hill North",16,16,16,16,16,16


In [39]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [40]:
# Analyze Each Neighborhood

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
# examine the new dataframe size
Toronto_onehot.shape

(780, 204)

In [42]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0
1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0
2,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0
4,Church and Wellesley,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0
5,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0
6,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.010,0.000000,0.060000,0.00,0.04,0.01,0.000000,0.0
7,"Dufferin, Dovercourt Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0
8,"Forest Hill West, Forest Hill North",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0
9,"Garden District, Ryerson",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00,0.000000,0.000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.0


In [43]:
# confirm the new size
Toronto_grouped.shape

(38, 204)

In [44]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                        venue  freq
0           Martial Arts Dojo   1.0
1                 Yoga Studio   0.0
2  Modern European Restaurant   0.0
3                   Locksmith   0.0
4                      Lounge   0.0


----Business Reply Mail Processing Centre 969 Eastern----
                        venue  freq
0                        Park  0.33
1                 Pizza Place  0.33
2           Mobile Phone Shop  0.33
3  Modern European Restaurant  0.00
4                   Locksmith  0.00


----Central Bay Street----
            venue  freq
0        Pharmacy  0.25
1     Pizza Place  0.25
2     Coffee Shop  0.25
3  Discount Store  0.25
4     Yoga Studio  0.00


----Christie----
               venue  freq
0               Park   0.5
1  Food & Drink Shop   0.5
2     Lingerie Store   0.0
3          Locksmith   0.0
4             Lounge   0.0


----Church and Wellesley----
            venue  freq
0     Pizza Place  0.29
1  Discount Store  0.14
2    Intersection  0.14
3     C

#### Let's put all that into a pandas dataframe

In [45]:
# a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
import numpy as np
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Martial Arts Dojo,Women's Store,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,Business Reply Mail Processing Centre 969 Eastern,Mobile Phone Shop,Park,Pizza Place,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,Central Bay Street,Pharmacy,Pizza Place,Coffee Shop,Discount Store,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,Christie,Park,Food & Drink Shop,Farmers Market,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
4,Church and Wellesley,Pizza Place,Coffee Shop,Sandwich Place,Chinese Restaurant,Intersection,Discount Store,Women's Store,Diner,Empanada Restaurant,Electronics Store


#### Cluster Neighborhoods

In [47]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 4, 0, 0, 0, 0, 0, 0])

In [48]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,0,Mexican Restaurant,Electronics Store,Breakfast Spot,Pizza Place,Intersection,Rental Car Location,Medical Center,Women's Store,Eastern European Restaurant,Dumpling Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,0,Café,College Stadium,General Entertainment,Skating Rink,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Pharmacy,Breakfast Spot,Sandwich Place,Bubble Tea Shop,Pizza Place,Grocery Store,Gym Pool
3,M4E,East Toronto,The Beaches,43.786947,-79.385975,0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714,2,Martial Arts Dojo,Women's Store,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [49]:
# Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'],\
                                  Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

In [50]:
# Cluster 1
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Mexican Restaurant,Electronics Store,Breakfast Spot,Pizza Place,Intersection,Rental Car Location,Medical Center,Women's Store,Eastern European Restaurant,Dumpling Restaurant
1,Downtown Toronto,0,Café,College Stadium,General Entertainment,Skating Rink,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,Downtown Toronto,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Pharmacy,Breakfast Spot,Sandwich Place,Bubble Tea Shop,Pizza Place,Grocery Store,Gym Pool
3,East Toronto,0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
5,Downtown Toronto,0,Pharmacy,Pizza Place,Coffee Shop,Discount Store,Women's Store,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
7,Downtown Toronto,0,Construction & Landscaping,Airport,Park,Playground,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
8,West Toronto,0,Grocery Store,Park,Shopping Mall,Bank,Diner,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
9,Downtown Toronto,0,Skating Rink,Beer Store,Park,Cosmetics Shop,Pharmacy,Curling Ice,Dance Studio,Women's Store,Dog Run,Ethiopian Restaurant
10,West Toronto,0,Trail,Health Food Store,Pub,Women's Store,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
11,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Grocery Store,Fruit & Vegetable Store,Pizza Place,Liquor Store


In [51]:
# Cluster 2
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,Downtown Toronto,1,Baseball Field,Women's Store,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [52]:
# Cluster 3
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,Martial Arts Dojo,Women's Store,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [53]:
# Cluster 4
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Downtown Toronto,3,Empanada Restaurant,Pizza Place,Women's Store,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [54]:
# Cluster 5
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Park,Food & Drink Shop,Farmers Market,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
